# Определение возраста покупателей

## Исследовательский анализ данных

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')

In [ ]:
labels.info()

<div class="alert alert-info" role="alert">
     Вывод: Датасет содержит два столбца file_name и по 7591 строк.Это фотографии и соотвестсвующие им данные о возрасте людей

In [ ]:
labels.head()

In [ ]:
labels['real_age'].unique()

<div class="alert alert-info" role="alert">
     Вывод: Аномальные заначения возраста отсуствуют

In [ ]:
labels.isna().sum()

<div class="alert alert-info" role="alert">
     Вывод: Пропуски отсуствуют

In [ ]:
labels.boxplot() 
plt.show()

<div class="alert alert-info" role="alert">
     Вывод: В среднем возраст людей на фото равен 30 годам

In [ ]:
labels.plot(kind='hist', figsize=(12, 6),bins = 40)
plt.title ('Распеределение возраста по выборке ')
plt.xlabel ('Возраст')
plt.ylabel ('Количество строчек соответсвующего возраста')
plt.legend()

<div class="alert alert-info" role="alert">
     Вывод: больше вссего избражений с людьми от  18 до 35 лет

In [ ]:
labels.shape

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 

<div class="alert alert-info" role="alert">
     Вывод: Мы видим, что многие изображения недостаточно  высокого качества, иногда снимки сделаны не в анфас или  часть лица закрыта  предметом(жевательная резинка, микрофон). Это может повлиять на качество работы модели.

## Обучение модели

Перенесите сюда код обучения модели и её результат вывода на экран.


(Код в этом разделе запускается в отдельном GPU-тренажёре, поэтому оформлен не как ячейка с кодом, а как код в текстовой ячейке)

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error 

def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')
    
    datagen = ImageDataGenerator(
        validation_split=0.25,
        horizontal_flip=True,
        rescale=1./255)
    train_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        seed=12345)
 
    return train_gen_flow
    
def load_test(path):
    labels = pd.read_csv(path + 'labels.csv')
    test_datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255)
    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)
 
    return test_gen_flow


def create_model(input_shape):
    optimizer = Adam(lr=0.0001)
    backbone = ResNet50(input_shape=(224,224,3),
                    weights='imagenet',
                    include_top=False) 
 
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer=optimizer, loss='mse',
                  metrics=['mae'])
    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model

In [ ]:
Train for 356 steps, validate for 119 steps
Epoch 1/20
2022-07-11 18:32:39.941199: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-11 18:32:42.126651: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
356/356 - 74s - loss: 196.1863 - mae: 10.2413 - val_loss: 543.3729 - val_mae: 18.2420
Epoch 2/20
356/356 - 39s - loss: 82.8269 - mae: 6.9252 - val_loss: 153.6239 - val_mae: 9.4462
Epoch 3/20
356/356 - 39s - loss: 56.7158 - mae: 5.7295 - val_loss: 78.5125 - val_mae: 6.6077
Epoch 4/20
356/356 - 39s - loss: 42.6512 - mae: 4.9981 - val_loss: 77.6264 - val_mae: 6.5696
Epoch 5/20
356/356 - 38s - loss: 33.8848 - mae: 4.4137 - val_loss: 92.7339 - val_mae: 7.2504
Epoch 6/20
356/356 - 39s - loss: 23.9111 - mae: 3.7632 - val_loss: 66.5872 - val_mae: 6.1902
Epoch 7/20
356/356 - 39s - loss: 19.6859 - mae: 3.3724 - val_loss: 69.9524 - val_mae: 6.1824
Epoch 8/20
356/356 - 39s - loss: 16.7656 - mae: 3.1210 - val_loss: 77.3779 - val_mae: 6.7963
Epoch 9/20
356/356 - 39s - loss: 15.3829 - mae: 2.9879 - val_loss: 68.8236 - val_mae: 6.3156
Epoch 10/20
356/356 - 39s - loss: 13.1031 - mae: 2.7666 - val_loss: 63.6125 - val_mae: 5.9492
Epoch 11/20
356/356 - 39s - loss: 12.8669 - mae: 2.6965 - val_loss: 63.2555 - val_mae: 6.0004
Epoch 12/20
356/356 - 39s - loss: 13.0365 - mae: 2.7061 - val_loss: 62.1264 - val_mae: 5.8932
Epoch 13/20
356/356 - 38s - loss: 11.1509 - mae: 2.5411 - val_loss: 64.0888 - val_mae: 5.9069
Epoch 14/20
356/356 - 39s - loss: 10.5313 - mae: 2.4766 - val_loss: 63.5779 - val_mae: 6.1005
Epoch 15/20
356/356 - 39s - loss: 10.1908 - mae: 2.4370 - val_loss: 60.6169 - val_mae: 5.7361
Epoch 16/20
356/356 - 39s - loss: 9.8868 - mae: 2.3954 - val_loss: 60.0284 - val_mae: 5.8039
Epoch 17/20
356/356 - 39s - loss: 9.7215 - mae: 2.3790 - val_loss: 76.5985 - val_mae: 6.6765
Epoch 18/20
356/356 - 39s - loss: 10.3380 - mae: 2.4548 - val_loss: 60.6996 - val_mae: 5.7048
Epoch 19/20
356/356 - 39s - loss: 10.4458 - mae: 2.4403 - val_loss: 66.9873 - val_mae: 6.2403
Epoch 20/20
356/356 - 39s - loss: 9.8914 - mae: 2.3451 - val_loss: 62.0239 - val_mae: 5.9326

## Анализ обученной модели

- MAE на валидационной выборке равноа 5.9326, следовательно поставленная задача (MAE модели не больше 8) выполнена 
- Для обучения использована нейронная сеть ResNett50
- Алгоритмом оптимизации выступала адаптивная оценка c параметром optimizer = Adam(lr=0.0001). 
- Обучение проходило на 10 эпохах.

## Чек-лист

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Исследовательский анализ данных выполнен
- [x]  Результаты исследовательского анализа данных перенесены в финальную тетрадь
- [x]  MAE модели не больше 8
- [x]  Код обучения модели скопирован в финальную тетрадь
- [x]  Результат вывода модели на экран перенесён в финальную тетрадь
- [x]  По итогам обучения модели сделаны выводы